In [1]:
import torch
from transformers import LongformerForSequenceClassification
from transformers import Trainer, TrainingArguments
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=2).to(device)
batch_size = 16
# check cuda availability
torch.cuda.is_available()
torch.cuda.current_device()
# load dataset
train = torch.load("../data/train_change_64")
test = torch.load("../data/test_change_64")
train.set_format("torch",
                            columns=["input_ids", "attention_mask", "label"])
logging_steps = len(train["train"]) // batch_size
training_args = TrainingArguments(output_dir="./results",
                                  num_train_epochs=3,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True,
                                  log_level="error")
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}
trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=train['train'],
                  eval_dataset=test['train'])
trainer.train()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()